# Predicting Parkinson's using EMD features and XGBoost

## Importing libraries 

In [10]:
from tqdm import tqdm # Imports `tqdm` to create a progress bad in Python

import pandas as pd # Imports the pandas library for tabular data
import numpy as np # Imports the numpy library for mathematical operations and linear algebra

import sklearn.metrics # Import metrics from sklearn
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, GridSearchCV # Import functions for cross validation

import xgboost as xgb # Import XGBoost for gradient boosting